In [ ]:
import numpy as np 
import h5py 
import matplotlib.pyplot as plt
import math
import random

a=h5py.File("mnist_traindata.hdf5",'r') 
b=h5py.File("mnist_testdata.hdf5",'r') 
xdata_test=b["xdata"][:]/255
labels_test=b["ydata"][:]


xdata_train = a["xdata"][:]/255
labels_train= a["ydata"][:]
X=np.hstack((xdata_train,labels_train))
a=np.take(X,np.random.permutation(X.shape[0]),axis=0,out=X)

xdata_train = a[:,0:784]
print(xdata_train.shape)
labels_train = a[:,784:794]

X_train = xdata_train[0:50000,:]
X_val = xdata_train[50000:60000,:]
Y_train = labels_train[0:50000, : ]
Y_val = labels_train[50000:60000, :]

lr=0.00015
error=0
error1=0

w1=np.random.rand(100,784)
b1=np.random.rand(100,500)

w2=np.random.rand(10,100) 
b2=np.random.rand(10,500)

def relu(x): 
    lst=[]
    for i in range(x.shape[0]): 
        if x[i]>0:
            lst.append(np.around(x[i],decimals=1))
        else:
            lst.append(0)
    return np.asarray(lst).reshape(100,1)


def relugrad(x):
    lst=[]
    for i in range(len(x)):
        if  x[i][0]>0:
            lst.append(1)
        elif x[i][0] < 0:
            lst.append(0)
        else :
            lst.append(random.randint(0,1))
    return np.asarray(lst).reshape(100,1)

def relu1(x): 
    lst=[]
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i][j]>0:
                lst.append(x[i][j])
            else:
                lst.append(0)
    return np.asarray(lst).reshape(100,500)


def relugrad1(x):
    lst=[]
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if  x[i][j]>0:
                lst.append(1)
            elif x[i][j] < 0:
                lst.append(0)
            else :
                lst.append(random.randint(0,1))
    return np.asarray(lst).reshape(100,500)

def smax(x): 
    lst=[] 

    for i in range(x.shape[0]):
        den=0
        for j in range(x.shape[1]):
            den=den+np.exp(x[i][j] - np.max(x[i]))
        for j in range(x.shape[1]): 
            num=np.exp(x[i][j] - np.max(x[i]))
            ans=(num/den) 
            lst.append(ans)
    return np.asarray(lst).reshape(500,10) 


for j in range(50):
    if j==20 or j==40:
        lr=lr/2
    acc=0
    acc1=0
    print("Epoch: ",j)
    for i in range(0,49500,500):
        x=X_train[i:i+500].T
        a1=relu1((w1@x) +b1 )
        at2=smax((w2@a1+b2).T)
        a2=at2.T
        del2 = a2-(Y_train[i:i+500].T)
        del1 = np.multiply(relugrad1(((w1@x) +b1)),(w2.T@del2))
        
        w1 = w1 - lr*(del1@x.T)
        b1 = b1 - lr*(del1)
        w2 = w2 - (lr*(del2@a1.T))
        b2 = b2 - lr*(del2)
        
        for k in range(500):
            if np.argmax(at2[k])==np.argmax(Y_train[i+k]):
                acc=acc+1
    acc=acc/50000
    print("Training accuracy is ",acc)
    plt.figure(1)
    curve1=plt.scatter(j,(acc),c='r',s=2)
    for i in range(0,9500,500):
        a11=relu1((w1@X_val[i:i+500].T + b1))
        a22=smax(((w2@a11) +b2).T)
        for k in range(500):
            if (np.argmax(a22[k])==np.argmax(Y_val[i+k])):
                acc1=acc1+1
    acc1=acc1/10000
    curve2=plt.scatter(j,(acc1),c='b',s=2)
    print("Validation accuracy is ",acc1)
    
curve1.set_label("Training accuracy")
curve2.set_label("Validation accuracy")
plt.title("Learning rate of 0.00015")
plt.legend() 
plt.show()